In [ ]:
# Derin Öğrenmenin Temelleri Proje.
# Defect Detection in Industrial Products.
# PCB

# Tespit Edebileceklerimiz
"""
0 – mouse_bite:
PCB kenarında fare ısırığı gibi küçük çentikler.

1 – spur:
İstenmeyen ince bakır çıkıntısı (kıl gibi uzantı).

2 – missing_hole:
Açılması gereken delik/via’nın hiç açılmamış olması.

3 – short:
İki bakır hattın kazara birleşip kısa devre oluşturması.

4 – open_circuit:
Bir bakır hattın kopması; bağlantının kesilmesi.

5 – spurious_copper:
Kart üzerinde gereksiz, şemaya ait olmayan bakır parçaları.
"""

'\n0 – mouse_bite:\nPCB kenarında fare ısırığı gibi küçük çentikler.\n\n1 – spur:\nİstenmeyen ince bakır çıkıntısı (kıl gibi uzantı).\n\n2 – missing_hole:\nAçılması gereken delik/via’nın hiç açılmamış olması.\n\n3 – short:\nİki bakır hattın kazara birleşip kısa devre oluşturması.\n\n4 – open_circuit:\nBir bakır hattın kopması; bağlantının kesilmesi.\n\n5 – spurious_copper:\nKart üzerinde gereksiz, şemaya ait olmayan bakır parçaları.\n'

In [1]:
# LINUX ORTAMINDAYIZ

!uname -a


Linux 32bbf861e55a 6.6.105+ #1 SMP Thu Oct  2 10:42:05 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
!nvidia-smi

Sun Dec 28 08:17:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             54W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
from google.colab import drive
drive.mount('/content/drive') # drive'mizi colab'a bağlıyoruz.

Mounted at /content/drive


In [5]:
# Drive'dan /content'e komple kopyala
!cp -r "/content/drive/MyDrive/Burcu Hoca Proje/Project/pcb-defect-dataset" "/content/pcb-defect-dataset"


^C


In [6]:
import yaml

yaml_yolu = "/content/pcb-defect-dataset/data.yaml"

with open(yaml_yolu, "r") as f:
    data = yaml.safe_load(f)

# Drive yerine content dizinini gösterelim
data["path"] = "/content/pcb-defect-dataset"
data["train"] = "train"
data["val"] = "val"
data["test"] = "test"

with open(yaml_yolu, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("✅ data.yaml güncellendi!")
print(open(yaml_yolu).read())


✅ data.yaml güncellendi!
path: /content/pcb-defect-dataset
train: train
val: val
test: test
names:
  0: mouse_bite
  1: spur
  2: missing_hole
  3: short
  4: open_circuit
  5: spurious_copper



In [7]:
%cd /content
!git clone https://github.com/iMoonLab/yolov13.git
%cd yolov13

# requirements.txt
with open("requirements.txt", "w") as f:
    f.write("""torch>=2.2.0
torchvision>=0.17.0
torchaudio>=2.2.0
tqdm
PyYAML
numpy
opencv-python>=4.8.0
pillow>=10.0.0
matplotlib
scipy
ultralytics>=8.3.63
supervision>=0.22.0
onnx==1.14.0
onnxruntime-gpu>=1.17.1
albumentations>=1.3.1
psutil
gradio""")

!pip install -r requirements.txt
!pip install -e .


/content
Cloning into 'yolov13'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 527 (delta 135), reused 116 (delta 116), pack-reused 229 (from 2)
Receiving objects: 100% (527/527), 29.91 MiB | 37.04 MiB/s, done.
Resolving deltas: 100% (159/159), done.
/content/yolov13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8

In [2]:
import shutil, os

base = "/content/yolov13/ultralytics/cfg/models/v13/yolov13.yaml"
target_dir = os.path.dirname(base)

for variant in ["n", "s", "l", "x"]:
    new_yaml = os.path.join(target_dir, f"yolov13{variant}.yaml")
    shutil.copy(base, new_yaml)
    with open(new_yaml, "r") as f:
        lines = f.readlines()
    with open(new_yaml, "w") as f:
        f.write(f"# Auto-generated {variant} model\nscale: \"{variant}\"\n")
        f.writelines(lines)
    print("✅ Created:", new_yaml)


✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13n.yaml
✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13s.yaml
✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13l.yaml
✅ Created: /content/yolov13/ultralytics/cfg/models/v13/yolov13x.yaml


In [3]:
# Drive bağlantısı
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

from ultralytics import YOLO

# Model
model = YOLO("/content/yolov13/ultralytics/cfg/models/v13/yolov13s.yaml")

# --- A100 80GB ayarlaması-
results = model.train(
    data="/content/pcb-defect-dataset/data.yaml",
    epochs=50,
    imgsz=960,

     # otomatik batch size
    batch=-1,

    device="cuda",

    # 2. Hızlandırma Ayarları
    workers=16,      # CPU GPU'ya veri yetiştirmek için kullanacak,çekirdek sayısı
    cache=True,      # RAM'den oku

   # 3. Drive kaydetme
    project="/content/drive/MyDrive/YOLO_Egitimleri",
    name="pcb_yolo13s_A100_Finalin_finalinin_finali",
    save=True,
    save_period=1,   # Her epoch'ta kaydet

    # Optimizasyon Ayarların
    optimizer="SGD", # optimizasyon algoritmamız
    lr0=0.01,  # learning rateyi dalgalı şekilde yürütür
    momentum=0.937,  # modelin önceki adımlarla bağlı olmasını sağlar
    cos_lr=True,   # learning rateyi dalgalı şekilde yürütür
    close_mosaic=10,  # Son 10 epochta 4 farklı resim üzerinden eğitim metodunu kapatır
    patience=20, # erken durdurma,ancak geniş optimizasyon aralığında
    amp=True # hesaplamaları hassas yapması için
)


FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
New https://pypi.org/project/ultralytics/8.3.241 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov13/ultralytics/cfg/models/v13/yolov13s.yaml, data=/content/pcb-defect-dataset/data.yaml, epochs=50, time=None, patience=20, batch=-1, imgsz=960, save=True, save_period=1, cache=True, device=cuda, workers=16, project=/content/drive/MyDrive/YOLO_Egitimleri, name=pcb_yolo13s_A100_Finalin_finalinin_finali, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=Non

100%|██████████| 10.1M/10.1M [00:00<00:00, 229MB/s]


AMP: checks passed ✅


train: Scanning /content/pcb-defect-dataset/train/labels.cache... 8534 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8534/8534 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

      50/50        56G      1.851      1.922      1.571         43        960: 100%|██████████| 225/225 [03:22<00:00,  1.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 15/15 [00:08<00:00,  1.71it/s]

                   all       1066       2164      0.962       0.46      0.492      0.297       0.29



50 epochs completed in 2.969 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Finalin_finalinin_finali/weights/last.pt, 18.7MB
Optimizer stripped from /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Finalin_finalinin_finali/weights/best.pt, 18.7MB

Validating /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Finalin_finalinin_finali/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLOv13s summary: 535 layers, 9,003,241 parameters, 0 gradients, 20.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 15/15 [00:09<00:00,  1.50it/s]


                   all       1066       2164      0.966      0.459      0.492      0.296       0.29
            mouse_bite        183        372      0.967      0.466        0.5      0.293      0.293
                  spur        179        369      0.979      0.387      0.418      0.234      0.235
          missing_hole        165        331      0.997       0.45      0.459      0.359      0.298
                 short        201        411       0.96      0.521      0.545      0.301      0.307
          open_circuit        178        359      0.938      0.513      0.553      0.314      0.321
       spurious_copper        160        322      0.958      0.416      0.477      0.274      0.283
Speed: 0.3ms preprocess, 5.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO_Egitimleri/pcb_yolo13s_A100_Finalin_finalinin_finali


In [11]:
import torch, gc
# gpu memory boşalt
def clear_gpu_memory():
    print("GPU memory'sini boşalt")
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("✅ Kalan kullanım:",
          f"{torch.cuda.memory_allocated()/1024**2:.1f} MB / {torch.cuda.get_device_properties(0).total_memory/1024**2:.1f} MB")

clear_gpu_memory()


GPU memory'sini boşalt
✅ Kalan kullanım: 448.3 MB / 81221.8 MB


In [ ]:
# RAM boşaltmak için,gerekli durumlarda.

import os
os.kill(os.getpid(), 9)


In [4]:
# CPU analizi,workers için

!lscpu | grep 'Model name'
!lscpu | grep 'CPU(s):'

Model name:                              Intel(R) Xeon(R) CPU @ 2.20GHz
CPU(s):                                  12
NUMA node0 CPU(s):                       0-11
